In [28]:
import pandas as pd
import PyPDF2
import re
import json

from transformers import AutoModelForCausalLM, AutoModelForSeq2SeqLM, AutoTokenizer, pipeline, BitsAndBytesConfig

import os
import torch
from langchain import PromptTemplate
from langchain.chains import SimpleSequentialChain, SequentialChain
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.llms.huggingface_pipeline import HuggingFacePipeline

import bitsandbytes

from PyPDF2 import PdfFileReader
import lxml.etree as ET
import pypandoc

In [29]:
# Utility functions
PC_PATH = "home/utente/Downloads/Codice penale.pdf"
REMOTE_PATH = "./Codice penale.pdf"

# !!! Write a customized regex to extract the text without words like avv. n. etc.
def extract_text_from_pdf(pdf_path):
    text = ""
    with open(pdf_path, 'rb') as file:
        reader = PyPDF2.PdfReader(file)
        for page_num in range(len(reader.pages)):
            page = reader.pages[page_num]
            text += page.extract_text()
    return text    

def extract_text_from_xml(xml_path):
    tree = ET.parse(xml_path)
    root = tree.getroot()
    return ET.tostring(root, encoding='unicode', method='text')

def extract_text_from_rtf(rtf_path):
    return pypandoc.convert_file(rtf_path, 'plain', format='rtf')

def write_to_file(filename, content):
    with open(filename, 'w') as f:
        f.write(content)

def read_from_file(filename):
    with open(filename, 'r') as f:
        return f.read()
    
def split_text(text, pattern):
    parts = re.split(pattern, text, flags=re.MULTILINE)
    
    parts = [part for part in parts if part]
    
    if not re.match(pattern, parts[0]):
        parts = parts[1:]
    
    return parts

# 1 Extracting quiz data from local files

In [3]:
# Extract questions from the PDF with quiz

def extract_questions_answers(text):
    qa_pattern = re.compile(r'(\d+)\. (.+?)\n(A\) .+?)\n(B\) .+?)\n(C\) .+?)\n', re.DOTALL)
    matches = qa_pattern.findall(text)
    
    data = []
    for match in matches:
        question_number, question, answer_a, answer_b, answer_c = match
        
        # Remove newline characters and clean up answers
        question = question.replace('\n', ' ')
        answer_a = answer_a.replace('\n', ' ').replace('A) ', '')
        answer_b = answer_b.replace('\n', ' ').replace('B) ', '')
        answer_c = answer_c.replace('\n', ' ').replace('C) ', '')
        
        data.append([question, answer_a, answer_b, answer_c])
    
    return data

PC_PATH = "home/utente/Downloads/03_diritto_amministrativo.pdf"
REMOTE_PATH = "./03_diritto_amministrativo.pdf"
text = extract_text_from_pdf(REMOTE_PATH)
qa_data = extract_questions_answers(text)

df_pdf = pd.DataFrame(qa_data, columns=['Domanda', 'Risposta 1', 'Risposta 2', 'Risposta 3'])
df_pdf.to_csv('quiz_pdf.csv', index=False)

df_pdf.head()


,Domanda,Risposta 1,Risposta 2,Risposta 3
0,"Dal punto di vista degli effetti, le autorizza...",Consentono ad un soggetto di non adempiere ad ...,Permettono di esercitare facoltà preesistenti.,Accertano l'esistenza dei presupposti richiest...
1,Il Capo II della l. n. 241/1990 è riservato al...,"Accerta d'ufficio i fatti, disponendo il compi...",Non è mai competente alla valutazione della su...,É solo competente all'indizione delle conferen...
2,Con riferimento al riesame con esito demolitor...,Quanto a competenza a disporla spetta all'orga...,Non comparta in nessun caso l'obbligo di provv...,Quanto a competenza a disporla non spetta in n...
3,Ai sensi dell'art. 80 C.p.a. come avviene la p...,Deve essere presentata istanza di fissazione d...,Deve essere presentata istanza di fissazione d...,Deve essere presentato nuovamente il ricorso e...
4,Entro quale termine le parti devono proporre r...,Nel termine di sessanta giorni decorrente dall...,Nel termine di novanta giorni decorrente dalla...,Nel termine di centoventi giorni decorrente da...


In [4]:
# Extract questions from the JSON file

PC_PATH = "/home/utente/Downloads/domande.json"
REMOTE_PATH = "domande.json"
with open(REMOTE_PATH, 'r') as file:
    data = json.load(file)

questions = []
answers1 = []
answers2 = []
answers3 = []

# Iterate over each question object in the JSON data
for item in data:
    question = item['question']
    answers = item['answers']
    
    # Extract each answer and its correctness
    correct_answer = None
    incorrect_answers = []
    for answer in answers:
        if answer['right']:
            correct_answer = answer['answer']
        else:
            incorrect_answers.append(answer['answer'])
    
    answers1.append(correct_answer)
    answers2.append(incorrect_answers[0])
    answers3.append(incorrect_answers[1])
    
    questions.append(question)

df_json = pd.DataFrame({
    'Domanda': questions,
    'Risposta 1': answers1,
    'Risposta 2': answers2,
    'Risposta 3': answers3
})

df_json.to_csv('quiz_json.csv', index=False)

df_json.head()


,Domanda,Risposta 1,Risposta 2,Risposta 3
0,Ai sensi dell'art. 240 c.p. è sempre ordinata ...,dei beni che costituiscono il profitto del rea...,dei servizi e degli strumenti informatici o do...,delle cose che costituiscono il movente del reato
1,"Ai sensi dell'art. 266 c.p. è punito, se il fa...",fa a militari l'apologia di fatti contrari al ...,istiga gli incaricati di pubblico servizio a v...,istiga i funzionari pubblici a disobbedire ai ...
2,La pena della multa ex art. 24 c.p. consiste n...,non inferiore a euro 50 né superiore a euro 50...,non inferiore a euro 10 né superiore a euro 50...,non inferiore a euro 25 né superiore a euro 20...
3,Ai sensi dell'art. 7 c.p. è punito secondo la ...,Delitti commessi da pubblici ufficiali a servi...,Delitti contro la personalità del Presidente d...,Delitti di contraffazione della marchiatura de...
4,Tra le pene accessorie per i delitti ex art. 1...,la decadenza o la sospensione dall'esercizio d...,l'estinzione del rapporto di impiego subordina...,l'inabilitazione dagli uffici direttivi delle ...


In [5]:
# Merge the two dataframes and clean up the data

df_merged = pd.concat([df_pdf, df_json], ignore_index=True)
df_merged.to_csv('quiz_merged_orig.csv', index=False)

# Delete rows which don't contain a number (a law reference)
df_merged = df_merged[df_merged['Domanda'].apply(lambda x: bool(re.search(r'\d', x)))]
df_merged = df_merged.drop_duplicates(subset='Domanda')

# Add an index colum at the beginning of the dataframe
df_merged.insert(0, 'Index', range(1, 1 + len(df_merged)))

df_merged.to_csv('quiz_merged.csv', index=False)
df_merged.head()

,Index,Domanda,Risposta 1,Risposta 2,Risposta 3
1,1,Il Capo II della l. n. 241/1990 è riservato al...,"Accerta d'ufficio i fatti, disponendo il compi...",Non è mai competente alla valutazione della su...,É solo competente all'indizione delle conferen...
3,2,Ai sensi dell'art. 80 C.p.a. come avviene la p...,Deve essere presentata istanza di fissazione d...,Deve essere presentata istanza di fissazione d...,Deve essere presentato nuovamente il ricorso e...
4,3,Entro quale termine le parti devono proporre r...,Nel termine di sessanta giorni decorrente dall...,Nel termine di novanta giorni decorrente dalla...,Nel termine di centoventi giorni decorrente da...
7,4,A norma di quanto dispone l'art. 133 del C.p.a...,Alla giurisdizione esclusiva del giudice ammin...,Alla giurisdizione esclusiva del giudice ordin...,Alla giurisdizione esclusiva del TAR del Lazio.
8,5,Consacrando a livello costituzionale i princip...,"Moneta, tutela del risparmio e mercati finanzi...",Tutela e sicurezza del lavoro.,"Produzione, trasporto e distribuzione nazional..."


# 2-3 Use of LLMs to extract informations from the quiz

In [ ]:
# Extract laws from a question
# Function to quantize and load models
def load_quantized_model(model_name):
    bnb_config = BitsAndBytesConfig(
                                load_in_4bit=True,
                                bnb_4bit_use_double_quant=True,
                                bnb_4bit_quant_type="nf4",
                                bnb_4bit_compute_dtype=torch.bfloat16,
                               )
        
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForCausalLM.from_pretrained(model_name, quantization_config=bnb_config, device_map="cuda")
    return model, tokenizer

# Extract laws from a question
def extract_laws_from_question(model, tokenizer, quiz_list):
    answers = []
    model_pipeline = pipeline("text-generation", model=model, tokenizer=tokenizer)

    question_prompt = (
        "Ti sottoporrò una domanda di un quiz sul mondo legislativo. "
        "Estraimi tutte i riferimenti a leggi che trovi e generare un JSON con i seguenti campi per ogni legge trovata: "
        "{numero della legge, testo della legge, link alla pagina web}. GENERA SOLAMENTE IL JSON, NESSUN TESTO AGGIUNTIVO."
    )

    #for index, row in quiz_list.iterrows():
    for index in range(3):
        row = quiz_list.iloc[index]
        context_prompt = row['Domanda']
        full_prompt = f"{question_prompt} Context: {context_prompt}"

        answer = model_pipeline(full_prompt, max_length=512, do_sample=True, top_p=0.95, num_return_sequences=1)
        answers.append(answer[0]['generated_text'])

        print(f"<---- Question {index} ---->\n")
        print(f"{full_prompt}")
        print("<---- Answer ---->\n")
        print(f"{answer}")

    return answers

# List of the used models
models = {
    #"Saul": {'model_name': 'Equall/Saul-7B-Instruct-v1', 'context_window': 1024}, #Modello addestrato su testi legali
    "Meta-Llama": {'model_name': 'meta-llama/Meta-Llama-3-8B-Instruct', 'context_window': 8000},
    #"Falcon-7B": {'model_name': 'tiiuae/falcon-7b-instruct', 'context_window': 512},
    #"Mixtral-8x22B": {'model_name': 'mistralai/Mixtral-8x22B-Instruct-v0.1', 'context_window': 1024},
    #"Minerva-3B": {'model_name': 'sapienzanlp/Minerva-3B-base-v1.0', 'context_window': 512}, # Modello italiano della Sapienza
    #"deepset/roberta-base-squad2" : {'model_name': 'deepset/roberta-base-squad2', 'context_window': 512} # Modello per il question answering
}

df = pd.read_csv('quiz_merged.csv')
output_models = []
output_answers = []

# Use each model to extract laws from the questions
for model_data_key in models.keys():
    model_data = models[model_data_key]
    print(f"Model: {model_data['model_name']}")

    model, tokenizer = load_quantized_model(model_data["model_name"])

    tmp_answers = extract_laws_from_question(model, tokenizer, df)
    
    output_answers.extend(tmp_answers)
    output_models.extend([model_data["model_name"]] * len(tmp_answers))

# Convert the dictionary to a DataFrame
df_extracted_laws = pd.DataFrame({
    'Model': output_models,
    'Question Index': list(range(len(output_answers))),
    'Extracted Laws': output_answers
})

print(df_extracted_laws)


In [19]:
# print the first element of the last column
df_extracted_laws['Extracted Laws'][0]

'Ti sottoporrò una domanda di un quiz sul mondo legislativo. Estraimi tutte i riferimenti a leggi che trovi e generare un JSON con i seguenti campi per ogni legge trovata: {numero della legge, testo della legge, link alla pagina web}. GENERA SOLAMENTE IL JSON, NESSUN TESTO AGGIUNTIVO. Context: Il Capo II della l. n. 241/1990 è riservato alla regolamentazione della figura del  responsabile del procedimento, ovvero del  soggetto al quale è affidato il delicato ruolo di autorità di guida di ciascun procedimento  amministrativo. Esso: 1) coordina e dirige il procedimento, 2) ne esercita la funzione di autorità di guida, 3) ne tiene conto delle esigenze di celerità e di efficacia, 4) ne coordina l\'attività degli uffici e dei dipendenti incaricati, 5) ne esercita la funzione di autorità di controllo e di garanzia. \n```json\n[\n  {\n    "numero della legge": "241/1990",\n    "testo della legge": "Capo II della l. n. 241/1990",\n    "link alla pagina web": "https://www.gazzettaufficiale.it/e

# 4 - Extract laws from raw files

In [36]:
def extract_cited_laws(text, model, tokenizer):
    # Assuming the model is fine-tuned for text extraction tasks or has the capability to handle such queries.
    inputs = tokenizer.encode("Extract cited laws from the following text: " + text, return_tensors="pt", max_length=512, truncation=True)
    outputs = model.generate(inputs, max_length=512, num_return_sequences=1)
    decoded_output = tokenizer.decode(outputs[0], skip_special_tokens=True)
    
    cited_laws = decoded_output.split("\n")
    return cited_laws

def extract_text_from_file(file_path, file_type):
    if file_type == "pdf":
        text = extract_text_from_pdf(file_path)
    elif file_type == "xml":
        text = extract_text_from_xml(file_path)
    elif file_type == "rtf":
        text = extract_text_from_rtf(file_path)
    else:
        raise ValueError("Unsupported file type")
    return text

# Function to split text into chunks
def split_text(text, max_chunk_size=7000, chunk_overlap=100):
    text_splitter = RecursiveCharacterTextSplitter(separators=[
        "\n\n",
    ],
    chunk_size=max_chunk_size, 
    chunk_overlap=chunk_overlap)
    
    return text_splitter.split_text([text])

In [38]:

# Initialize the model for extracting cited laws
bnb_config = BitsAndBytesConfig(
                                load_in_4bit=True,
                                bnb_4bit_use_double_quant=True,
                                bnb_4bit_quant_type="nf4",
                                bnb_4bit_compute_dtype=torch.bfloat16,
                               )
        
tokenizer = AutoTokenizer.from_pretrained("meta-llama/Meta-Llama-3-8B-Instruct")
model = AutoModelForCausalLM.from_pretrained("meta-llama/Meta-Llama-3-8B-Instruct", quantization_config=bnb_config, device_map="cuda")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Loading checkpoint shards: 100%|██████████| 4/4 [00:48<00:00, 12.08s/it]


In [37]:
# Extract laws from a file


filesPathToAnalyze = [REMOTE_PATH]
citedLaws = []

for filePath in filesPathToAnalyze:
    fileText = extract_text_from_file(filePath, re.search(r'\.([a-zA-Z0-9]+)$', REMOTE_PATH).group(1).lower())
    textChunks = split_text(fileText, 7500)
    for chunk in textChunks:
        citedLaws.append(extract_cited_laws(chunk, model, tokenizer))
    
print(citedLaws)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Loading checkpoint shards: 100%|██████████| 4/4 [00:54<00:00, 13.67s/it]


AttributeError: 'Document' object has no attribute 'text'

# 5 - Generation of a similar law